In [2]:
!pip install pyTelegramBotAPI

     |████████████████████████████████| 71kB 2.4MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.1-cp36-none-any.whl size=50883 sha256=3b2115518d6eef753f32d637e1e00523841911801ecef15a80f5d99c7ef379af
  Stored in directory: /root/.cache/pip/wheels/cd/b2/2c/eac6af3343b21f907123ce013d20ad5ad70c2c3731072d98bf
Successfully built pyTelegramBotAPI


In [0]:
import telebot
import os
import random
import pickle

token = "1292604678:AAEZfEn_8PVtoNcKYJ6VOOtwOWaB3aGr_TI" #токен бота

handler = str.lower #то что будет обрабатывать сообщения

bot = telebot.TeleBot(token)

#загружаем температуры пользователей
if os.path.exists("temperatures.save"):
  with open("temperatures.save", "rb") as f:
    temperatures = pickle.load(f)
else:
  temperatures = {}

#сообщение при старте
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Используйте /get_p  и /set_p x для того, чтобы узнать и поменять вероятность ответа')
    if  message.from_user.id not in temperatures:
      temperatures[message.from_user.id] = 0.5


#изменение температуры
@bot.message_handler(commands=['set_p'])
def change_t_message(message):
    try:
      t = float(message.text.split()[1])
    except:
      bot.send_message(message.chat.id, 'Вероятность должна быть числом между 0 и 1')
      return

    if (t >= 0) and (t <= 1):
      temperatures[message.from_user.id] = t
      bot.send_message(message.chat.id, f'Теперь вероятность - {t}')
      with open("temperatures.save", "wb") as f:
        pickle.dump(temperatures, f)
    else:
      bot.send_message(message.chat.id, 'Вероятность должна быть числом между 0 и 1')

#пишет температуру
@bot.message_handler(commands=['get_p'])
def change_t_message(message):
    if message.from_user.id not in temperatures:
      temperatures[message.from_user.id] = 0.5
    t = temperatures[message.from_user.id]
    bot.send_message(message.chat.id, t)

#обработчик сообщений
@bot.message_handler(content_types=['text'])
def send_text(message):
  if message.from_user.id == 268917708:
    bot.send_message(message.chat.id, "Царин клоун")
  else:
    if message.from_user.id not in temperatures:
      temperatures[message.from_user.id] = 0.5
    if random.random() > (1-temperatures[message.from_user.id]):
      bot.send_message(message.chat.id, handler(message.text))

bot.polling()